In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup  ##파싱

In [2]:
#html_id list
crawling_list=[
    'Text1898',
    'Text1900',
    'Text3194',
    'Text3196',
    'Text_975',
    'Text2033',
    'Cell1518',
    'Text1915',
    'Text1910',
    'Text1911',
    'Text1912',
    'Text4164',
    'Text4165',
    'Text2051',
    'Text2053',
    'Text2055',
    'Text2057',
    'Text4125',

    'Cell689',
    'Cell690',
    'Cell691',
    'Cell692',
    'Cell693',
    'Cell694',
    'Cell695',
    'Cell696',
    'Cell697',
    'Cell698',

    'Cell_892',
    'Cell_893',

    'Cell699',
    'Cell700',

    'Text2118',
    'Text2122',
    'Text2123',
    'Text2124',

    'Text2146',
    'Text2148',
    'Text2150',
    'Text2152',
    'Text2154',

    'Text2144'
]

In [3]:
#html id 별 설명 항목
crawling_list_exp=[
    '채권명',
    '투자자모집시작일',
    '투자인원',
    '모집금액',
    '안심펀드',
    '연금리',
    '투자현황',
    '대출목적',
    '상품',
    '만기',
    '대출금액',
    '상환방식',
    '월상환액',
    '8percent신용등급',
    '8percent예측불량률',
    'KCB신용등급',
    'KCB예측불량률',
    '주요사항',
    '담보건수',
    '담보금액',
    '은행보험학자금건수',
    '은행보험학자금금액',
    '저축은행캐피탈카드론건수',
    '저축은행캐피탈카드론금액',
    '현금서비스건수',
    '현금서비스금액',
    '대부건수',
    '대부금액',
    'P2P건수',
    'P2P금액',
    '총합건수',
    '총합금액',
    '월소득',
    'RTI',
    '월가처분소득',
    'LTI',
    '직장',
    '업종',
    '근무기간',
    '고용형태',
    '연소득',
    '심사총론',
]

In [4]:
#기본 크롤링 대상 url , url maximum값 입력
BASE_URL='https://8percent.kr/investment/detail/'
CRAWLING_NUM=1177

In [5]:
# 지정된 url에서 한 채권 data 수집
def one_bond_crawling(BASE_URL):
    response=requests.get(BASE_URL)
    dom=BeautifulSoup(response.content, "html.parser")
    url_number=BASE_URL.replace('https://8percent.kr/investment/detail/','') #html 뒤에 들어가는 숫자만 추출
    #DOM => Document Object Model
    id_list=[]
    text_list=[]
    for i in crawling_list:
        try:
            a=dom.select_one('div#'+str(i)).text
            text_list.append(a)
            id_list.append(i)
        except AttributeError:
            text_list.append('None')
            id_list.append(i)
    df_text=pd.DataFrame(text_list) #채권의 수치들
    df_id=pd.DataFrame(id_list) #html id
    df_column_name=pd.DataFrame(crawling_list_exp) #html id 설명들 최종 column이 될 것들
    df_ond_bond=pd.concat([df_column_name,df_id,df_text], axis=1) #상기 df 합쳐줌
    df_ond_bond.columns = ['column_name','id','txt'] #df column 이름 지정해줌
    df_ond_bond['txt'].replace(regex=True,inplace=True,to_replace=' ',value='') #띄어쓰기 지우기
    df_ond_bond['txt'].replace(regex=True,inplace=True,to_replace='\n',value='') #쓸모없는 글 지우기
    indexed_df_ond_bond = df_ond_bond.set_index('column_name') #인덱스명 바꾸기
    del indexed_df_ond_bond['id'] #필요없는 column 지우기
    new_indexed_df_ond_bond=indexed_df_ond_bond.rename(columns = {'txt':url_number}).transpose() #column 이름 바꾸고 transpose
    
    return new_indexed_df_ond_bond

In [6]:
#예시
pd.set_option('display.max_columns', 50)
one_bond_crawling('https://8percent.kr/investment/detail/525')

column_name,채권명,투자자모집시작일,투자인원,모집금액,안심펀드,연금리,투자현황,대출목적,상품,만기,대출금액,상환방식,월상환액,8percent신용등급,8percent예측불량률,KCB신용등급,KCB예측불량률,주요사항,담보건수,담보금액,은행보험학자금건수,은행보험학자금금액,저축은행캐피탈카드론건수,저축은행캐피탈카드론금액,현금서비스건수,현금서비스금액,대부건수,대부금액,P2P건수,P2P금액,총합건수,총합금액,월소득,RTI,월가처분소득,LTI,직장,업종,근무기간,고용형태,연소득,심사총론
525,제473호:타기관대출상환,15.12.15,0,0,1.54%,13.51%,투자취소됨,타기관대출상환,개인신용,24개월,"1,000만원",원리금균등상환,47만원,C-,4.49%,7등급(621),8.48%,연체경험미보유,0건,0만원,3건,"9,760만원",2건,"2,198만원",0건,0만원,0건,0만원,0건,0만원,5건,"11,958만원",486만원,28.0%,109만원,205.0%,대기업,광고업,1.4년,정규직,"5,832만원",본대출자는39세남성으로현재광고업에근무중입니다.8퍼센트대출금으로카드론1건을상환할예정입...


In [7]:
#상기 정의한 함수로 전체 채권 data 수집
def whole_bond(start,end):
    df=pd.DataFrame()
    for i in range(start,end):
        one_bond=one_bond_crawling(BASE_URL+ str(i))
        df = pd.concat([df, one_bond], ignore_index=False)
    return df

In [8]:
#58번 전까지는 url포멧이 다름. 56번은 같음
raw_data=whole_bond(0,CRAWLING_NUM)
raw_data['url_number'] = raw_data.index

In [9]:
raw_data

column_name,채권명,투자자모집시작일,투자인원,모집금액,안심펀드,연금리,투자현황,대출목적,상품,만기,대출금액,상환방식,월상환액,8percent신용등급,8percent예측불량률,KCB신용등급,KCB예측불량률,주요사항,담보건수,담보금액,은행보험학자금건수,은행보험학자금금액,저축은행캐피탈카드론건수,저축은행캐피탈카드론금액,현금서비스건수,현금서비스금액,대부건수,대부금액,P2P건수,P2P금액,총합건수,총합금액,월소득,RTI,월가처분소득,LTI,직장,업종,근무기간,고용형태,연소득,심사총론,url_number
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1
2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2
3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,3
4,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,4
5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5
6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,6
7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,7
8,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,8
9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,9


업데이트모듈만들기

In [10]:
# 기존 데이터 중에 펀딩중인 채권만 뽑아내기
def now_funding_bond(raw_data):
    df=raw_data[raw_data['투자현황']=='None'] # 현재 투자가능한 채권(빈 채권도 속해있음)
    df_now_funding=df[df['채권명']!='None'] # 빈채권 없애주기 =>순수히 현재 모집중인 채권
    return df_now_funding

In [11]:
df_now_funding=now_funding_bond(raw_data)

In [12]:
# 기존 데이터 중에 펀딩중인 채권만 크롤링
def specific_bond(df_now_funding):
    df=pd.DataFrame()
    for i in df_now_funding.index:
        one_bond=one_bond_crawling(BASE_URL+ str(i))
        df = pd.concat([df, one_bond], ignore_index=False)
    return df

In [13]:
# now funding인 채권 업데이트
def update_bond(raw_data):
    df_need_to_update=specific_bond(df_now_funding)
    df_need_to_update['url_number']=df_now_funding['url_number']
    raw_data_updated=pd.concat([raw_data.drop(df_now_funding.index),df_need_to_update])
    return raw_data_updated

In [14]:
update_bond(raw_data)

column_name,채권명,투자자모집시작일,투자인원,모집금액,안심펀드,연금리,투자현황,대출목적,상품,만기,대출금액,상환방식,월상환액,8percent신용등급,8percent예측불량률,KCB신용등급,KCB예측불량률,주요사항,담보건수,담보금액,은행보험학자금건수,은행보험학자금금액,저축은행캐피탈카드론건수,저축은행캐피탈카드론금액,현금서비스건수,현금서비스금액,대부건수,대부금액,P2P건수,P2P금액,총합건수,총합금액,월소득,RTI,월가처분소득,LTI,직장,업종,근무기간,고용형태,연소득,심사총론,url_number
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1
2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2
3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,3
4,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,4
5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,5
6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,6
7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,7
8,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,8
9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,9


In [15]:
update_bond(raw_data).to_excel('raw_data.xlsx','raw_data')

In [16]:
%ls -la

total 13240
drwxr-xr-x  20 GS  staff      680  4  7 17:12 ./
drwxr-xr-x+ 33 GS  staff     1122  4  2 13:45 ../
-rw-r--r--@  1 GS  staff     8196  3 29 21:49 .DS_Store
drwxr-xr-x  11 GS  staff      374  3 29 16:45 .ipynb_checkpoints/
-rw-r--r--   1 GS  staff   146805  3 16 13:46 2016.03.11-8percent_crawling_test.ipynb
-rw-r--r--   1 GS  staff    15024  4  7 17:12 2016.03.16-8percent_crawling_module.ipynb
-rw-r--r--   1 GS  staff   501638  3 29 14:13 2016.03.16-8percent_preprocessing_test.ipynb
-rw-r--r--   1 GS  staff    91731  4  7 17:07 2016.03.17-8percent_preprocessing_module.ipynb
-rw-r--r--   1 GS  staff  4838230  3 29 14:15 2016.03.18-8percent_kmeans_test1.ipynb
-rw-r--r--   1 GS  staff    28519  4  7 17:07 2016.03.21-8percent_similarity_module.ipynb
-rw-r--r--   1 GS  staff   175406  4  7 15:25 2016.03.21-8percent_similarity_test.ipynb
-rw-r--r--   1 GS  staff    64361  3 29 16:11 2016.03.23-8percent_kmeans_test2.ipynb
-rw-r--r--   1 GS  staff    92478  4  7 17:05 2016.03.29-8per